In [5]:
import requests 
from bs4 import BeautifulSoup as bs 
import re
from datetime import datetime
import pandas as pd

In [2]:
start_time = datetime.now()

In [3]:
r = requests.get('https://www.canada.ca/en/health-canada/services/consumer-product-safety/cosmetics/cosmetic-ingredient-hotlist-prohibited-restricted-ingredients/hotlist.html',verify = "Y:\\Python Notebooks\\Chubb_CA_2016.cer") 

prohibited_register_number = []
restricted_register_number = []


soup = bs(r.content) 
data = []
table_body = soup.findAll('table')[0]
string = table_body.get_text()
if re.search("[0-9]+-[0-9]+-[0-9]+", string) and  "CAS" in string:
    d = re.findall("[0-9]+-[0-9]+-[0-9]+", string)
    prohibited_register_number.extend(d)
    
table_body = soup.findAll('table')[1]
string = table_body.get_text()
if re.search("[0-9]+-[0-9]+-[0-9]+", string) and  "CAS" in string:
    d = re.findall("[0-9]+-[0-9]+-[0-9]+", string)
    restricted_register_number.extend(d)

In [10]:
df1 = pd.DataFrame(prohibited_register_number,columns=['CAS'])
df1['type'] = 'Prohibited'
df2 = pd.DataFrame(restricted_register_number,columns=['CAS'])
df2['type'] = 'Restricted'
frames = [df1, df2]
dataset = pd.concat(frames)
dataset.rename(columns = {'CAS':'CAS Number'}, inplace = True)
dataset.head(2)

,CAS Number,type
0,106-88-7,Prohibited
1,106-99-0,Prohibited


In [15]:
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Not prohibited'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)
result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.type.fillna(result.value, inplace=True)
del result['value']
result.head(2)

,CAS Number,type
0,106-88-7,Prohibited
1,106-99-0,Prohibited


In [16]:
result['Jurisdiction'] = 'Canada'
result['RegulatoryAgency'] = 'Health Canada'
result['TypeOfRegulation'] = 'Use'
result['Criteria'] = 'Use in cosmetics'


result['Description'] = 'Prohibited in cosmetics products'
result.loc[result['type'] == 'Restricted', 'Description'] = 'Restricted in cosmetics products'
result.loc[result['type'] == 'Not prohibited', 'Description'] = 'Not prohibited or restricted in cosmetics products'


result['HazardClassification'] = 'High'
result.loc[result['type'] == 'Restricted', 'HazardClassification'] = 'Moderate'
result.loc[result['type'] == 'Not prohibited', 'HazardClassification'] = 'Low'


result['CauseOfClassification'] = 'Prohibited by Health Canada'
result.loc[result['type'] == 'Restricted', 'CauseOfClassification'] = 'Restricted by Health Canada1'
result.loc[result['type'] == 'Not prohibited', 'CauseOfClassification'] = 'Not Prohibited by Health Canada1'


result['SourceName'] = 'Health Canada'
result['SourceURL'] = 'https://www.canada.ca/en/health-canada/services/consumer-product-safety/cosmetics/cosmetic-ingredient-hotlist-prohibited-restricted-ingredients/hotlist.html'

result = result.drop(['type'], axis=1)
result.rename(columns = {'CAS':'CAS Number'}, inplace = True)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,106-88-7,Canada,Health Canada,Use,Use in cosmetics,Prohibited in cosmetics products,High,Prohibited by Health Canada,Health Canada,https://www.canada.ca/en/health-canada/service...
1,106-99-0,Canada,Health Canada,Use,Use in cosmetics,Prohibited in cosmetics products,High,Prohibited by Health Canada,Health Canada,https://www.canada.ca/en/health-canada/service...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [17]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:04:12.900353
